In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 60.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pulp

# Define the given 9 DOE combinations (factor values for mu_set and temperature).
# These correspond to a 2-factor Doehlert design: 5 levels for mu_set and 3 for temperature (with center point repeated).
factor_values = np.array([
    [0.135, 31.0],  # Combination 1
    [0.135, 31.0],  # Combination 2:
    [0.135, 31.0],  # Combination 3:
    [0.16, 31.0],  # Combination 4:
    [0.1475, 33.0],  # Combination 5:
    [0.11, 31.0],  # Combination 6:
    [0.1225, 29.0],  # Combination 7:
    [0.1475, 29.0],  # Combination 8:
    [0.1225, 33.0]   # Combination 9:

])



# Problem Parameters
# Number of DOE combinations (nDoE) and factors (mu_set and temperature)
nComb = len(factor_values)
nFactors = len(factor_values[1])
# Assume up to 27 intensified experiments (9 parameters at 3 stages each) as an upper bound (will be minimized by objective)
# niDoE <= nDoE
nStages = 3
nExp = nComb * nStages # upper max number of experiments limit nExp is non intensified nDoE




# Set the max step change allowed between sequential stages for each factor (C7 thresholds)
# Can be changed by the ueser
delta_f_max_mu   = 0.03   # mu_set difference <= 0.02
delta_f_max_temp = 2    # temperature difference <= 2 °C

# Set the min step change allowed between sequential stages for each factor (C8 thresholds)
# Can be changed by the ueser

delta_f_min_mu   = 0.01
delta_f_min_temp = 1

# Decision Variables
# Binary variable x[i][j][k] = 1 if DOE combination j is used at stage k of experiment i.
x = pulp.LpVariable.dicts('x', (range(1, nExp+1), range(1, nComb+1), range(1, nStages+1)),
                          lowBound=0, upBound=1, cat='Binary')

# Additional aid binary variables for C8 (Big-M formulation)
z = pulp.LpVariable.dicts('z', (range(1, nExp+1), range(1, nFactors+1)), lowBound=0, upBound=1, cat='Binary')
q = pulp.LpVariable.dicts('q', (range(1, nStages), range(1, nExp+1), range(1, nFactors+1)), lowBound=0, upBound=1, cat='Binary')

# Optimization Problem
# Objective minimize the niDoE from nDoE subsequently the cost
problem = pulp.LpProblem("iDoE_Planner", pulp.LpMinimize)

# Objective Function (Eq. 2–3): Minimize a weighted cost that penalizes using additional experiments.
# The cost weight increases with experiment index i, so using higher-index experiments adds more cost.
# w[i] = (i / (nDoE + 1))^3

weights = (np.arange(1, nExp+1) / float(nComb + 1)) ** 3

# NumPy array of weights for experiments 1 to nExp (max number of experiments nExp is non intensified nDoE )
problem += pulp.lpSum(weights[i-1] * x[i][j][k]
                      for i in range(1, nExp+1)
                      for j in range(1, nComb+1)
                      for k in range(1, nStages+1)), "Minimize_Experiments_Cost"

# Constraints

# C1 (Eq. 4): Only one DOE combination can be used per stage of an experiment:
# For each experiment i and each stage k, sum of x[i,j,k] over all j <= 1.
for i in range(1, nExp+1):
    for k in range(1, nStages+1):
        problem += pulp.lpSum(x[i][j][k] for j in range(1, nComb+1)) <= 1, f"C1_one_comb_per_stage(i{i}_k{k})"

# C2 (Eq. 5): A DOE combination appears at most once at any given stage position across all experiments:
# For each combination j and stage k, sum of x[i,j,k] over all experiments i <= 1.
for j in range(1, nComb+1):
    for k in range(1, nStages+1):
        problem += pulp.lpSum(x[i][j][k] for i in range(1, nExp+1)) <= 1, f"C2_unique_at_stage(j{j}_k{k})"

# C3 (Eq. 6): A DOE combination may be used at most twice in a single experiment
# For each experiment i and combination j, sum of x[i,j,k] over stages k <= 2.
for i in range(1, nExp+1):
    for j in range(1, nComb+1):
        problem += pulp.lpSum(x[i][j][k] for k in range(1, nStages+1)) <= 2, f"C3_max2_per_expt(i{i}_j{j})"

# C4 (Eq. 7): A DOE combination may be used at most twice across all experiments (global repetition limit)
# For each combination j, sum of x[i,j,k] over all i and k <= 2.
for j in range(1, nComb+1):
    problem += pulp.lpSum(x[i][j][k] for i in range(1, nExp+1) for k in range(1, nStages+1)) <= 2, f"C4_max2_global(j{j})"

# C5 (Eq. 8): Every DOE combination must be used at least once in the intensified plan
# For each combination j, sum of x[i,j,k] over all i and k >= 1.
for j in range(1, nComb+1):
    problem += pulp.lpSum(x[i][j][k] for i in range(1, nExp+1) for k in range(1, nStages+1)) >= 1, f"C5_cover_all(j{j})"

# C6 (Eq. 9): Weighted stage repetition – Enforce strategic repetition of combinations across dierent stages with a desired pattern
# Introduce weights s_k for each stage k and a minimum total score t_j for each combination j
# I choose s = [1, 1, 1] (equal weight for each of 3 stages as suggused in the paper) and t_j = 2 for all j, meaning each combo should appear in at least 2 stages in total.
# (For center-point combinations which were replicated in the original DOE, t_j can be set lower to allow flexibility
s = {1: 1, 2: 1, 3: 1}            # weight for each stage
t = {j: 2 for j in range(1, nComb+1)}
# If needed, reduce t for the center-point since it was replicated three times in the original DOE
t[1] = t[2] = t[3] = 1


# Apply the weighted repetition constraint: for each combination j, sum_{i,k}(s_k * x[i,j,k]) >= t_j.
for j in range(1, nComb+1):
    problem += pulp.lpSum(s[k] * x[i][j][k]
                           for i in range(1, nExp+1)
                           for k in range(1, nStages+1)) >= t[j], f"C6_weighted_repetition(j{j})"

# C7 (Eq. 10–12): Sequential change limits – limit the step change in mu_set and temperature between sequential stages
# For each experiment i, stage transition k→k+1, and factor l (upper limit):
#    | f_value(j,l) * x[i,j,k] – f_value(j',l) * x[i,j',k+1] | <= delta f_max,l  (upper limit)
# I linearize the absolute difference with two constraints (one for positive and one for negative difference)
# Add function in the future
for i in range(1, nExp+1):
    for k in range(1, nStages):  # k and k+1 are consecutive stages
        # Difference in μ_set between stage k and k+1 <= 0.02
        problem += (pulp.lpSum(factor_values[j-1, 0] * x[i][j][k]   for j in range(1, nComb+1))
                    - pulp.lpSum(factor_values[j-1, 0] * x[i][j][k+1] for j in range(1, nComb+1))
                   ) <= delta_f_max_mu, f"C7_mu_diff(i{i}_stage{k}-{k+1})"
        problem += (pulp.lpSum(factor_values[j-1, 0] * x[i][j][k]   for j in range(1, nComb+1))
                    - pulp.lpSum(factor_values[j-1, 0] * x[i][j][k+1] for j in range(1, nComb+1))
                   ) >= -delta_f_max_mu, f"C7_mu_diff_neg(i{i}_stage{k}-{k+1})"
        # Difference in temperature between stage k and k+1 <= 2.2 °C
        problem += (pulp.lpSum(factor_values[j-1, 1] * x[i][j][k]   for j in range(1, nComb+1))
                    - pulp.lpSum(factor_values[j-1, 1] * x[i][j][k+1] for j in range(1, nComb+1))
                   ) <= delta_f_max_temp, f"C7_temp_diff(i{i}_stage{k}-{k+1})"
        problem += (pulp.lpSum(factor_values[j-1, 1] * x[i][j][k]   for j in range(1, nComb+1))
                    - pulp.lpSum(factor_values[j-1, 1] * x[i][j][k+1] for j in range(1, nComb+1))
                   ) >= -delta_f_max_temp, f"C7_temp_diff_neg(i{i}_stage{k}-{k+1})"

# C8 (Eq. 13–17): Minimum variation per experiment – enforce a minimum overall change in mu_set and temperature across each experiment
# This ensures each experiment spans a certain range in each factor, using a Big-M (logical OR) formulation
# delta f_min,l is the required total variation for factor l across the full experiment .

M = 1000  # Big M (a large constant, larger than any feasible LHS sum)
L = 500   # Big L (another large constant, M > L > 0)

for i in range(1, nExp+1):
    # For each experiment i and factor l (1=μ_set, 2=temperature), apply Eq. (14)–(17) constraints.
    for l, delta_f_min in [(1, delta_f_min_mu), (2, delta_f_min_temp)]:
        # Compute factor l differences for stage1→2 (diff1) and stage2→3 (diff2) in experiment i
        diff1 = pulp.lpSum(factor_values[j-1, l-1] * x[i][j][1] for j in range(1, nComb+1)) \
                - pulp.lpSum(factor_values[j-1, l-1] * x[i][j][2] for j in range(1, nComb+1))
        diff2 = pulp.lpSum(factor_values[j-1, l-1] * x[i][j][2] for j in range(1, nComb+1)) \
                - pulp.lpSum(factor_values[j-1, l-1] * x[i][j][3] for j in range(1, nComb+1))
        # Eq. (14) and (15) for k=1 (stage1 to stage2)
        problem += diff1 + M * z[i][l] + L * (q[1][i][l] + q[2][i][l]) >= delta_f_min,       f"C8_min_var_k1(i{i}_l{l})"
        problem += diff1 - M * z[i][l] + L * (q[1][i][l] + q[2][i][l]) >= delta_f_min - M,   f"C8_min_var_k1_alt(i{i}_l{l})"
        # Eq. (16) and (17) for k=2 (stage2 to stage3)
        problem += diff2 + M * z[i][l] - L * q[1][i][l] + L * q[2][i][l] >= delta_f_min - L,  f"C8_min_var_k2(i{i}_l{l})"
        problem += diff2 - M * z[i][l] - L * q[1][i][l] + L * q[2][i][l] >= delta_f_min - M - L, f"C8_min_var_k2_alt(i{i}_l{l})"

# Note: z[i][l] and q[k][i][l] are helper binaries to enforce that **at least one** stage transition per experiment i
# has a large enough difference in factor l (ensuring total variation >= delta f_min,l)
# They are added to the objective at zero cost.




# Solve the MILP problem
problem.solve(pulp.PULP_CBC_CMD(msg=False))

# Print output the optimized assignment of DOE points to each stage of each experiment
for i in range(1, nExp+1):
    experiment_used = False # Default not used X[i][j][k]
    assignment = [] # Reset per new experiment
    for k in range(1, nStages+1): # Loop through stages
        for j in range(1, nComb+1): # Loop through conditions
            if pulp.value(x[i][j][k]) == 1: # if used i.e. binary = 1
                experiment_used = True
                assignment.append(f"Stage {k}: Combo {j} (μ_set={factor_values[j-1,0]}, Temp={factor_values[j-1,1]}°C)") # add to result
    if experiment_used:
        print(f"Experiment {i}: " + ", ".join(assignment)) # Print result


Experiment 1: Stage 1: Combo 9 (μ_set=0.1225, Temp=33.0°C), Stage 2: Combo 6 (μ_set=0.11, Temp=31.0°C), Stage 3: Combo 6 (μ_set=0.11, Temp=31.0°C)
Experiment 2: Stage 1: Combo 7 (μ_set=0.1225, Temp=29.0°C), Stage 2: Combo 1 (μ_set=0.135, Temp=31.0°C), Stage 3: Combo 8 (μ_set=0.1475, Temp=29.0°C)
Experiment 3: Stage 1: Combo 5 (μ_set=0.1475, Temp=33.0°C), Stage 2: Combo 2 (μ_set=0.135, Temp=31.0°C), Stage 3: Combo 7 (μ_set=0.1225, Temp=29.0°C)
Experiment 4: Stage 1: Combo 3 (μ_set=0.135, Temp=31.0°C), Stage 2: Combo 9 (μ_set=0.1225, Temp=33.0°C), Stage 3: Combo 5 (μ_set=0.1475, Temp=33.0°C)
Experiment 5: Stage 1: Combo 4 (μ_set=0.16, Temp=31.0°C), Stage 2: Combo 8 (μ_set=0.1475, Temp=29.0°C), Stage 3: Combo 4 (μ_set=0.16, Temp=31.0°C)
